---
title: Nand2Knuck
---
https://www.nand2tetris.org/course

https://www.philipzucker.com/nand2tetris-chc/
https://www.philipzucker.com/nand2tetris-cpu/
https://www.philipzucker.com/nand2tetris-in-verilog-and-fpga-and-coq/
https://github.com/philzook58/nand2coq

In [6]:
Nandtable = [
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1],
    [0, 0, 1]
]

def test_nand():
    for x,y,z in Nandtable:
        assert not (x and y) == z

test_nand()

def fuzz_nand():
    for testnum in range(1000):
        x = random.randint()
        y = random.randint()
        z = nand(x, y)



SyntaxError: invalid syntax (3501921742.py, line 12)

In [1]:
from knuckledragger import lemma, axiom, define
from z3 import *

B = BoolSort()
x,y,z,w,u,v = Consts("x y z w u v", B)

CNand = Function("Nand", B,B,B,B)
nand_def = axiom(ForAll([x,y,z], CNand(x,y,z) == 
     Or(And(x == True,  y == True,  z == False),
        And(x == True,  y == False, z == True),
        And(x == False, y == True,  z == True),
        And(x == False, y == False, z == True))))


test1 = lemma(CNand(True,True,False), by=[nand_def])

nand_fun = lemma(ForAll([x,y,z], CNand(x,y,z) == (Not(And(x,y)) == z)), by=[nand_def])
nand_fun

&ctdot; &#8870;ForAll([x, y, z], Nand(x, y, z) == (Not(And(x, y)) == z))

In [2]:
test2 = lemma(CNand(True,True,True), by=[nand_def])

LemmaError: (Nand(True, True, True), 'Countermodel', [Nand = [(True, True, True) -> False,
         else ->
         Or(Not(Or(Not(Var(2)), Var(1), Not(Var(0)))),
            Not(Or(Var(2), Not(Var(1)), Not(Var(0)))),
            Not(Or(Not(Var(2)), Var(0), Not(Var(1)))),
            Not(Or(Not(Var(2)), Var(1), Var(0))))]])

In [5]:
CNot = Function("CNot", B, B, B)
not_def = ForAll([x,y], CNot(x,y) == CNand(x,x,y))

CNot, not_def = define("CNot", [x,y], CNand(x,x,y))
lemma(ForAll([x,y], CNot(x,y) == (Not(x) == y)), by=[nand_def, not_def])

not_fun = lemma(ForAll([x,y], CNot(x,y) == (Not(x) == y)), by=[nand_fun, not_def])


CAnd, and_def = define("CAnd", [x,y,z], Exists([w], And(CNand(x,y,w), CNot(w,z))))

cand_fun = lemma(ForAll([x,y,z], CAnd(x,y,z) == (And(x,y) == z)), by=[not_fun, nand_fun, and_def])


COr, or_def = define("COr", [x,y,z], Exists([w,u], And(CNot(x,w), CNot(y,u), 
                                                   CNand(w,u,z))))

cor_fun = lemma(ForAll([x,y,z], COr(x,y,z) == (Or(x,y) == z)), by=[not_fun, nand_fun, or_def])


# Chapter 2


In [ ]:
half_adder = define("HalfAdder", [a,b,c,s], 
    Or(
        And(a == False, b == False, c == False, s == False),
        And(a == False, b == True,  c == False, s == True),
        And(a == True,  b == False, c == False, s == True),
        And(a == True,  b == True,  c == True,  s == False)
    ))


# FullAdder


# RippleCarryAdder


# ALU


In [ ]:

#yada yada


flipflip == Function("D",B,B,B,B)
flip_flip_def = ForAll([x,y])



# Other
Calculating compilers
https://xavierleroy.org/courses/EUTypes-2019/


In [2]:
def Switch(*cases):
    if len(cases) == 1:
        return cases[0]
    assert len(cases) % 2 == 1
    return If(case[0], cases[1], Switch(*cases[2:]))

# Yeah, balancing all those parens isn't that bad.
# maybe this abstraction isn't worth it


#Switch(Aexpr.is_num(e), Aexpr.val(e),
#       Aexpr.is_add(e), Aexpr.val(e[0]), Aexpr.val(e[1])),
#       IntVal(42)
#       )

In [25]:
from z3 import *
def lemma(thm, by=[], admit=False):
    if admit:
        print("Admitting", thm)
    else:
        prove(Implies(And(by), thm))
    return thm

In [27]:

"""
Tiny helper. But I need more pretty fast.
Ah, but it is a bit odd to write, but cleaner to use to just attach stuff to this object.

def List(s : SortRef):
    L = Datatype(f"List_{s}")
    L.declare("nil")
    L.declare("cons", ("car", s), ("cdr", L))
    L = L.create()


    L.induct = 
    L.from_list =
    return L

"""
# https://coq.inria.fr/doc/master/stdlib/Coq.Lists.List.html
class List():
    def __init__(self, elt : SortRef, admit=False):
        L = Datatype(f"List_{elt}")
        L.declare("nil")
        L.declare("cons", ("car", elt), ("cdr", L))
        L = L.create()
        
        self.elt = elt 
        self.t = L
        
        """
        This is just bulk junk.
        I could maybe just subclass DatatypeSortRef?
        self.nil = L.nil
        self.cons = L.cons
        self.car = L.car
        self.cdr = L.cdr
        self.is_cons = L.is_cons
        self.
        """
        x,y,z,l = Consts("x y z l", L)
        a,b,c = Consts("a b c", elt)

        self.append = Function("append", L, L, L)
        self.append_def = ForAll([x, y], self.append(x, y) == If(L.is_nil(x), y,
                                                                 L.cons(L.car(x), self.append(L.cdr(x), y))))
        self.app_nil_l = lemma(ForAll([x], self.append(L.nil, x) == x), by=[self.append_def])
        self.app_nil_r = lemma(ForAll([x], self.append(x, L.nil) == x), by=[self.append_def, self.induct(lambda x : self.append(x, L.nil) == x)])
        self.append_assoc = lemma(ForAll([x,y,z], self.append(x, self.append(y,z)) == self.append(self.append(x,y), z)), admit=True)

        self.mem = Function("mem", L, elt, BoolSort())
        self.mem_def = ForAll([a, l], self.mem(l, a) == If(L.is_nil(l), False,
                                                        If(L.car(l) == a, True, self.mem(L.cdr(l), a))))
        
        self.rev = Function("rev", L, L)
        self.rev_def = ForAll([l], self.rev(l) == If(L.is_nil(l), l, 
                                                     self.append(self.rev(L.cdr(l)), L.cons(L.car(l), L.nil))))
        
        # map, filter
    def Const(self, name):
        # could I attach operator overloads? It'd be really nice.
        return Const(name, self.t)
        

    def induct(self, P):
        x = FreshConst(self.elt, prefix="x")
        l = FreshConst(self.t, prefix="l")
        return Implies(And(
                        P(self.t.nil),
                        ForAll([x,l], Implies(P(l), P(self.t.cons(x, l))))),
                       #-----------------------------------
                        ForAll([l], P(l)))
    def from_list(self,l : list[ExprRef]):
        if len(l) == 0:
            return self.sort.nil
        return self.sort.cons(l[0], self.from_list(l[1:]))
# Running over this should be sufficient, because List_Dummy has no properties
# This is a meta argument though.
List(DeclareSort("List_Dummy"))

proved
proved
Admitting ForAll([x, y, z],
       append(x, append(y, z)) == append(append(x, y), z))


IndentationError: expected an indented block after class definition on line 4 (3081262896.py, line 7)

First define arithmetic expressions

In [30]:

Aexpr = Datatype("Aexpr")
Aexpr.declare("num", ("val", IntSort()))
Aexpr.declare("add", ("left", Aexpr), ("right", Aexpr))
Aexpr = Aexpr.create()
n = Int("n")
e, e1, e2 = Consts('e e1 e2', Aexpr)
def induct_aexpr(P):
    return Implies(And(        
        ForAll([n],     P(Aexpr.num(n))),
        ForAll([e1,e2], 
               Implies(And(P(e1), P(e2)), 
                       P(Aexpr.add(e1, e2))))),
        #-------------------------------------
        ForAll([e], P(e)))

eval = Function("eval", Aexpr, IntSort())
eval_def = ForAll([e], eval(e) == If(Aexpr.is_num(e), Aexpr.val(e),
                                     eval(Aexpr.left(e)) + eval(Aexpr.right(e))))

In [38]:
op = Datatype("op")
op.declare("add")
op.declare("push", ("val", IntSort()))
op = op.create()

def Option(s : SortRef):
    O = Datatype(f"Option_{s}")
    O.declare("none")
    O.declare("some", ("val", s))
    O = O.create()
    return O

IntList = List(IntSort())
Stack = IntList.t

exec = Function("exec", op, IntList.t, IntList.t) # option?
o = Const("o", op)
l = Const("l", IntList.t)
car = IntList.t.car
cdr = IntList.t.cdr
cons = IntList.t.cons
exec_def = ForAll([o,l], exec(o,l) == If(op.is_push(o), cons(op.val(o), l),
                                      #If(op.is_add(o),  
                                         cons(car(l) + car(cdr(l)), cdr(cdr(l)))))
                                     # )

Prog = List(op)
p = Const("p", Prog.t)
exec_prog = Function("exec_prog", Prog.t, IntList.t, IntList.t)
exec_prog_def = ForAll([p,l], exec_prog(p, l) == If(Prog.t.is_nil(p), l,
                                                  exec_prog(Prog.t.cdr(p), exec(Prog.t.car(p), l)))
                        )

compile = Function("compile", Aexpr, Prog.t)
compile_def = ForAll([e], compile(e) == If(Aexpr.is_num(e), Prog.t.cons(op.push(Aexpr.val(e)), Prog.t.nil),
                                           Prog.append(compile(Aexpr.left(e)), 
                                           Prog.append(compile(Aexpr.right(e)), 
                                                       Prog.t.cons(op.add, Prog.t.nil)))))




SyntaxError: incomplete input (2888385662.py, line 38)

In [9]:
type(IntList)

z3.z3.DatatypeSortRef

In [ ]:
class Nat():
    def __init__(self):
        pass
class Pos():
    
# Any is not closed and therefor doesn't have an indeuction principle of proving something true forall x:Any, unless it uses pure first order reasoning.
class Any():
    def __init__(self):
        self.t = DeclareSort("Any")
        self.sorts = {} # mapping from thing to Any injector,projector, tester
    def add_sort(s):
        self.sorts[s] = 
    def inj(self, sort):
        return Function(f"inj_{sort}", sort, self.t)
    def is_sort(self, sort):
        is_sort_def = ForAll([x], is_sort(x) == Exists([t], self.inj(sort)(t) == x))
        Function(f"is_{sort}", self.t, BoolSort())
    def injective(self):
        pass

class Map():
    pass
class Set():
    pass
class Enum():
    pass
class Record():
    pass

Nand 2 tetris

Maybe that's too agressive. How about a calculator

Quoted aexpr. hmm.
```ocaml
type aexpr =
 | Int of int
 | Plus of aexpr * aexpr
 | Quote of aexpr
```



```lean
-- import Mathlib
import Mathlib.Data.Nat.Basic
import Mathlib.Tactic.Basic
import Mathlib.Tactic.LibrarySearch
-- import Std.Data.List.Basic
-- import Std.Data.Nat.Basic
--open Nat
--#print Nat.
#eval 1 + 1
inductive aexpr where
  | num : ℕ → aexpr
--| var : string → aexpr
  |  add : aexpr → aexpr → aexpr

def eval : aexpr → ℕ
  | .num n => n
  | .add e1 e2 => eval e1 + eval e2

inductive op where
  | add : op
  | push : ℕ → op

-- [@reducible]
abbrev Prog := List op
abbrev Stack := List ℕ

def exec (p : Prog) (s : Stack) : Stack :=
  match p with
  | [] => s
  | (op.add :: p') =>
    match s with
    | (n1 :: n2 :: s') => exec p' ((n1 + n2) :: s')
    | _ => exec p' s
  | (op.push n :: p') => exec p' (n :: s)

def compile : aexpr → Prog
  | .num n => [op.push n]
  | .add e1 e2 => (compile e1) ++ (compile e2) ++ [op.add]
#print List
lemma exec_append : forall p1 p2 s, exec (p1 ++ p2) s = exec p2 (exec p1 s) := by
  intros p1 p2
  induction p1 with
  | nil => simp [exec]
  | cons h t ih =>
    simp [exec]
    cases h with
    | add =>
      intros s
      cases s with
      | nil => simp [exec, ih]
      | cons n1 s' =>
        cases s' with
        | nil => simp [exec, ih]
        | cons n2 s'' =>
          simp [exec]
          rw [ih]
    | push n =>
      simp [exec]
      intros s
      rw [ih]

theorem exec_compile : ∀ e s, exec (compile e) s = eval e :: s := by
  intros e
  induction e with
  | num n => simp [compile, exec, eval]
  | add e1 e2 IHe1 IHe2 =>
    intros s
    --simp [eval, compile, exec_append, exec]
    simp [eval]
    have : exec [op.add] ((eval e2) :: (eval e1) :: s) = (eval e1 + eval e2) :: s := by
      simp [exec, add_comm]
    rw [<- this]
    rw [<- IHe1]
    rw [<- IHe2]
    rw [<- exec_append]
    simp [compile]
    rw [exec_append]

/-
inductive fib : Nat -> Nat -> Type where
  | base0 : fib 0 0
  | base1 : fib 1 1
  | step  : n > 1 -> fib (n-1) a
                 -> fib (n-2) b -> fib n (a+b)

-/

```

```lean
-- import Mathlib
import Mathlib.Data.Nat.Basic
import Mathlib.Tactic.Basic
import Mathlib.Tactic.LibrarySearch
-- import Std.Data.List.Basic
-- import Std.Data.Nat.Basic
--open Nat
--#print Nat.
#eval 1 + 1
inductive aexpr where
  | num : ℕ → aexpr
--| var : string → aexpr
  |  add : aexpr → aexpr → aexpr

def eval : aexpr → ℕ
  | .num n => n
  | .add e1 e2 => eval e1 + eval e2

inductive op where
  | add : op
  | push : ℕ → op

-- [@reducible]
abbrev Prog := List op
abbrev Stack := List ℕ

def exec (p : Prog) (s : Stack) : Stack :=
  match p with
  | [] => s
  | (op.add :: p') =>
    match s with
    | (n1 :: n2 :: s') => exec p' ((n1 + n2) :: s')
    | _ => exec p' s
  | (op.push n :: p') => exec p' (n :: s)

def compile : aexpr → Prog
  | .num n => [op.push n]
  | .add e1 e2 => (compile e1) ++ (compile e2) ++ [op.add]
#print List
lemma exec_append : forall p1 p2 s, exec (p1 ++ p2) s = exec p2 (exec p1 s) := by
  intros p1 p2
  induction p1 with
  | nil => simp [exec]
  | cons h t ih =>
    simp [exec]
    cases h with
    | add =>
      intros s
      cases s with
      | nil => simp [exec, ih]
      | cons n1 s' =>
        cases s' with
        | nil => simp [exec, ih]
        | cons n2 s'' =>
          simp [exec]
          rw [ih]
    | push n =>
      simp [exec]
      intros s
      rw [ih]

theorem exec_compile : ∀ e s, exec (compile e) s = eval e :: s := by
  intros e
  induction e with
  | num n => simp [compile, exec, eval]
  | add e1 e2 IHe1 IHe2 =>
    intros s
    --simp [eval, compile, exec_append, exec]
    simp [eval]
    have : exec [op.add] ((eval e2) :: (eval e1) :: s) = (eval e1 + eval e2) :: s := by
      simp [exec, add_comm]
    rw [<- this]
    rw [<- IHe1]
    rw [<- IHe2]
    rw [<- exec_append]
    simp [compile]
    rw [exec_append]

/-
inductive fib : Nat -> Nat -> Type where
  | base0 : fib 0 0
  | base1 : fib 1 1
  | step  : n > 1 -> fib (n-1) a
                 -> fib (n-2) b -> fib n (a+b)

-/
```